# Create your desired plots for statistical results and other helpful graphics

Gather necessary pre-requisites:

In [1]:
# import everything you need
import nilearn.plotting as ni_plt
import matplotlib.pyplot as plt
import nibabel as nb
import os, re
from sys import getsizeof  # function is not fully indicative -> look for better option

200418-07:49:23,949 nipype.utils INFO:
	 Running nipype version 1.5.0-dev (latest: 1.4.2)


In [2]:
# define useful functions
# define show-slices function
def show_slices(slices):
    """ Function to display row of image slices """
    fig, axes = plt.subplots(1, len(slices))
    for i, slice in enumerate(slices):
        axes[i].imshow(slice.T, cmap="gray", origin="lower")
# add suptitle via plt.suptitle

# check size of items and delete them when above a certain threshold
def check_itemsize(items):
    for i in items:
        # return size of object in bytes
        print("size of item", i, "is:", getsizeof(i))
        if getsizeof(i) > 10000: # think about dynamic coding for this
            del i
            print("deleted, object exceeded set threshold.")
        else:
            print("retained, object is sufficiently small.")

In [3]:
# specify relevant directories
VBM_dir = "/VBM/"
norm_dirs = [os.path.join(VBM_dir, normdir)
             for normdir in os.listdir(VBM_dir)
             if normdir.startswith("DARTEL_norm2mni")]

MReg_dirs = [os.path.join(normdir, regdir)
             for normdir in norm_dirs 
             for regdir in os.listdir(normdir)
             if re.match(r'.*SPM_M.*', regdir)]

MReg_ests = [os.path.join(regdir, estdir)
             for regdir in MReg_dirs 
             for estdir in os.listdir(regdir)
             if re.match(r'est.*', estdir)]

Contrast_dirs = sorted([os.path.join(estdir, contdir)
                        for estdir in MReg_ests 
                        for contdir in os.listdir(estdir)
                        if re.match(r'est.*Cont.*', contdir)])


In [4]:
print(Contrast_dirs)

['/VBM/DARTEL_norm2mni_c1/GM_SPM_MReg_ANCScale/estimate_MReg/estimate_Contrasts', '/VBM/DARTEL_norm2mni_c1/GM_SPM_MReg_PropScale/estimate_MReg/estimate_Contrasts', '/VBM/DARTEL_norm2mni_c2/WM_SPM_MReg_ANCScale/estimate_MReg/estimate_Contrasts', '/VBM/DARTEL_norm2mni_c2/WM_SPM_MReg_PropScale/estimate_MReg/estimate_Contrasts']


Get files

In [5]:
# make one dict of all post-FDR dirs sorted into lists by content
# [contrasts 1-4: dep>con, con>dep, male>female, young>old]

# specify number of spmT_images
spmT_img_len=4
stat_dirs_FDR={}

# enter content in alphabetically sorted order
stat_dirs_FDR["GM_ANCscale"] = sorted([os.path.join(contdir, f"FDR_spmT{contrast_index}")
                                  for contdir in Contrast_dirs 
                                  for contrast_index in range(1, (spmT_img_len + 1))
                                  if re.match(r'.*GM.*ANCScale.*', contdir)])

stat_dirs_FDR["GM_propscale"] = sorted([os.path.join(contdir, f"FDR_spmT{contrast_index}")
                                   for contdir in Contrast_dirs 
                                   for contrast_index in range(1, (spmT_img_len + 1))
                                   if re.match(r'.*GM.*PropScale.*', contdir)])

stat_dirs_FDR["WM_ANCscale"] = sorted([os.path.join(contdir, f"FDR_spmT{contrast_index}")
                                  for contdir in Contrast_dirs 
                                  for contrast_index in range(1, (spmT_img_len + 1))
                                  if re.match(r'.*WM.*ANCScale.*', contdir)])

stat_dirs_FDR["WM_propscale"] = sorted([os.path.join(contdir, f"FDR_spmT{contrast_index}")
                                       for contdir in Contrast_dirs 
                                       for contrast_index in range(1, (spmT_img_len + 1))
                                       if re.match(r'.*WM.*PropScale.*', contdir)])

# get the spmT images after FDR thresholding (show only surviving clusters)
stat_images_postFDR={}

for key in stat_dirs_FDR.keys():
    stat_images_postFDR[key] = sorted([os.path.join(stat_dirs_FDR[key][index], img)
                                       for index in range(0, len(stat_dirs_FDR[key]))
                                       for img in os.listdir(stat_dirs_FDR[key][index])
                                       if re.match(r'(^spmT.*\d{4}_thr.nii)', img)])

# define contrast names in order of creation (MReg model)
con_names = ["dep>con", "con>dep", "male>female", "young>old"]

# define the split GM and WM templates
DARTEL_templates = [VBM_dir + f"DARTEL_template/template_6_{i}.nii" for i in ["GM", "WM"]]

In [6]:
# control results 
print("FDR directories ...\n")
for k,v in stat_dirs_FDR.items():
    print(k,"\n",v,"\n",len(v))
    
print("FDR thresholded statistical images ...\n")
for k,v in stat_images_postFDR.items():
    print(k,"\n",v,"\n",len(v))

FDR directories ...

GM_ANCscale 
 ['/VBM/DARTEL_norm2mni_c1/GM_SPM_MReg_ANCScale/estimate_MReg/estimate_Contrasts/FDR_spmT1', '/VBM/DARTEL_norm2mni_c1/GM_SPM_MReg_ANCScale/estimate_MReg/estimate_Contrasts/FDR_spmT2', '/VBM/DARTEL_norm2mni_c1/GM_SPM_MReg_ANCScale/estimate_MReg/estimate_Contrasts/FDR_spmT3', '/VBM/DARTEL_norm2mni_c1/GM_SPM_MReg_ANCScale/estimate_MReg/estimate_Contrasts/FDR_spmT4'] 
 4
GM_propscale 
 ['/VBM/DARTEL_norm2mni_c1/GM_SPM_MReg_PropScale/estimate_MReg/estimate_Contrasts/FDR_spmT1', '/VBM/DARTEL_norm2mni_c1/GM_SPM_MReg_PropScale/estimate_MReg/estimate_Contrasts/FDR_spmT2', '/VBM/DARTEL_norm2mni_c1/GM_SPM_MReg_PropScale/estimate_MReg/estimate_Contrasts/FDR_spmT3', '/VBM/DARTEL_norm2mni_c1/GM_SPM_MReg_PropScale/estimate_MReg/estimate_Contrasts/FDR_spmT4'] 
 4
WM_ANCscale 
 ['/VBM/DARTEL_norm2mni_c2/WM_SPM_MReg_ANCScale/estimate_MReg/estimate_Contrasts/FDR_spmT1', '/VBM/DARTEL_norm2mni_c2/WM_SPM_MReg_ANCScale/estimate_MReg/estimate_Contrasts/FDR_spmT2', '/VBM/DARTE

Plot VBM results

In [7]:
# plot results only for any non-empty contrast after thresholding
# for GM and WM (separately for proportional and ANCOVA scaling)

for img_list,dir_list in zip(stat_images_postFDR.keys(),stat_dirs_FDR.keys()):
    for index in range(0, len(stat_images_postFDR[img_list])):
        index_nb = nb.load(stat_images_postFDR[img_list][index])
        # if there are any values above zero in the thresholded T-stat image, create figures
        if index_nb.dataobj[:,:,:][index_nb.dataobj[:,:,:] > 0].any():
            print(" Statistical image  ", stat_images_postFDR[img_list][index], "  contains significant values.")
            print("Proceeding with figure creation ...")
            stat_img_3D_GM = ni_plt.view_img(stat_images_postFDR[img_list][index],
                                               threshold=0, cut_coords=[-42, -16, 52],
                                               title=con_names[2])
            stat_map_zslice10_GM = ni_plt.plot_stat_map(stat_images_postFDR[img_list][index],
                                                          threshold=0, display_mode="z",
                                                          cut_coords=10, title=con_names[2])
            stat_map_xslice10_GM = ni_plt.plot_stat_map(stat_images_postFDR[img_list][index],
                                                          threshold=0, display_mode="x",
                                                          cut_coords=10, title=con_names[2])
            stat_map_yslice10_GM = ni_plt.plot_stat_map(stat_images_postFDR[img_list][index],
                                                          threshold=0, display_mode="y",
                                                          cut_coords=10, title=con_names[2])
            stat_img_3D_GM.save_as_html(os.path.join(stat_dirs_FDR[dir_list][index],
                                                     f"{stat_dirs_FDR[dir_list][index].rsplit('_', 1)[1]}_3Dimg_{dir_list.split('_', 1)[0]}.html"))
            stat_map_zslice10_GM.savefig(os.path.join(stat_dirs_FDR[dir_list][index],
                                                      f"{stat_dirs_FDR[dir_list][index].rsplit('_', 1)[1]}_statmap_Zslice10_{dir_list.split('_', 1)[0]}.png"))
            stat_map_xslice10_GM.savefig(os.path.join(stat_dirs_FDR[dir_list][index],
                                                      f"{stat_dirs_FDR[dir_list][index].rsplit('_', 1)[1]}_statmap_Xslice10_{dir_list.split('_', 1)[0]}.png"))
            stat_map_yslice10_GM.savefig(os.path.join(stat_dirs_FDR[dir_list][index],
                                                      f"{stat_dirs_FDR[dir_list][index].rsplit('_', 1)[1]}_statmap_Yslice10_{dir_list.split('_', 1)[0]}.png"))
        else:
            print("This image seems to be empty. Continuing with next image ...")


for img_list,dir_list in zip(stat_images_postFDR.keys(),stat_dirs_FDR.keys()):
    for index in range(0, len(stat_images_postFDR[img_list])):
        index_nb = nb.load(stat_images_postFDR[img_list][index])
        # if there are any values above zero in the thresholded T-stat image, create figures
        if index_nb.dataobj[:,:,:][index_nb.dataobj[:,:,:] > 0].any():
            print(" Statistical image  ", stat_images_postFDR[img_list][index], "  contains significant values.")
            print("Proceeding with figure creation ...")
            index_nb = nb.load(stat_images_postFDR[img_list][index])
            for temp in range(0, len(DARTEL_templates)):
                stat_map_zslice10_ontemplate_GM = ni_plt.plot_stat_map(stat_images_postFDR[img_list][index],
                                                                         threshold=0, display_mode="z",
                                                                         cut_coords=10, title=con_names[index],
                                                                         bg_img=DARTEL_templates[temp])
                stat_map_zslice10_ontemplate_GM.savefig(os.path.join(stat_dirs_FDR[dir_list][index],
                                                                     f"{stat_dirs_FDR[dir_list][index].rsplit('_', 1)[1]}_statmap_Zslice10_ontemplate_{dir_list.split('_', 1)[0]}.png"))
        else:
            print("This image seems to be empty. Continuing with next image ...")

This image seems to be empty. Continuing with next image ...
This image seems to be empty. Continuing with next image ...
This image seems to be empty. Continuing with next image ...
This image seems to be empty. Continuing with next image ...
This image seems to be empty. Continuing with next image ...
This image seems to be empty. Continuing with next image ...
 Statistical image   /VBM/DARTEL_norm2mni_c1/GM_SPM_MReg_PropScale/estimate_MReg/estimate_Contrasts/FDR_spmT3/spmT_0003_thr.nii   contains significant values.
Proceeding with figure creation ...


/opt/miniconda-latest/envs/neuro/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in greater
  


This image seems to be empty. Continuing with next image ...
This image seems to be empty. Continuing with next image ...
This image seems to be empty. Continuing with next image ...
This image seems to be empty. Continuing with next image ...
This image seems to be empty. Continuing with next image ...
This image seems to be empty. Continuing with next image ...
This image seems to be empty. Continuing with next image ...
 Statistical image   /VBM/DARTEL_norm2mni_c2/WM_SPM_MReg_PropScale/estimate_MReg/estimate_Contrasts/FDR_spmT3/spmT_0003_thr.nii   contains significant values.
Proceeding with figure creation ...


/opt/miniconda-latest/envs/neuro/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in greater
  


This image seems to be empty. Continuing with next image ...
This image seems to be empty. Continuing with next image ...
This image seems to be empty. Continuing with next image ...
This image seems to be empty. Continuing with next image ...
This image seems to be empty. Continuing with next image ...
This image seems to be empty. Continuing with next image ...
This image seems to be empty. Continuing with next image ...
 Statistical image   /VBM/DARTEL_norm2mni_c1/GM_SPM_MReg_PropScale/estimate_MReg/estimate_Contrasts/FDR_spmT3/spmT_0003_thr.nii   contains significant values.
Proceeding with figure creation ...


/opt/miniconda-latest/envs/neuro/lib/python3.7/site-packages/ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in greater


This image seems to be empty. Continuing with next image ...
This image seems to be empty. Continuing with next image ...
This image seems to be empty. Continuing with next image ...
This image seems to be empty. Continuing with next image ...
This image seems to be empty. Continuing with next image ...
This image seems to be empty. Continuing with next image ...
This image seems to be empty. Continuing with next image ...
 Statistical image   /VBM/DARTEL_norm2mni_c2/WM_SPM_MReg_PropScale/estimate_MReg/estimate_Contrasts/FDR_spmT3/spmT_0003_thr.nii   contains significant values.
Proceeding with figure creation ...
This image seems to be empty. Continuing with next image ...


Testing Zone

In [10]:
# plot medial slices of all con images pre-threshold

# get con files for GM Proportional Scaling
con_img = sorted([os.path.join(con_dir, con)
                  for con_dir in Contrast_dirs
                  for con in os.listdir(con_dir)
                  if re.match(r'^.*GM.*PropScale', con_dir)
                  and re.match(r'^con_\d{4}.nii', con)])

# check con images
print(con_img)

# loop plotting and plot saving for all con_img
con_img_plot = {}

for i in con_img:
    # images are centered at AC-PC line. don't need centre coordiantes -> take origin
    #dimensions = nb.load(i).header['dim'][1:4]
    con_img_plot[i] = ni_plt.plot_anat(i, display_mode='ortho', draw_cross=False, black_bg=True, cut_coords=[0,0,0])
    con_img_plot[i].savefig(os.path.join(i.rsplit('/', 1)[0], f"{i.rsplit('/', 1)[1][:-4]}"))

['/VBM/DARTEL_norm2mni_c1/GM_SPM_MReg_PropScale/estimate_MReg/estimate_Contrasts/con_0001.nii', '/VBM/DARTEL_norm2mni_c1/GM_SPM_MReg_PropScale/estimate_MReg/estimate_Contrasts/con_0002.nii', '/VBM/DARTEL_norm2mni_c1/GM_SPM_MReg_PropScale/estimate_MReg/estimate_Contrasts/con_0003.nii', '/VBM/DARTEL_norm2mni_c1/GM_SPM_MReg_PropScale/estimate_MReg/estimate_Contrasts/con_0004.nii']


[70 82 70]
